In [1]:
import pandas as pd
import os
import gc
import pickle
import numpy as np

from time import sleep

import gspread_dataframe as gs_df

os.chdir("c:/Users/rsilva/gdrive/projects/programming/automation")

# INITIALIZE DRIVER
from selenium import webdriver
from sapiens import LoginPage, KEYS, Page_sapiens
import functions
#import sistemas


% reload_ext autoreload
% autoreload 2

CPF = '33831640807'
PASS_1 = "Fer465023@ur_"
PASS_2 = "Fer465023"

### Loga Driver no Sapiens

In [4]:
def start_sapiens(cpf, senha, sapiens=None):   
   
    page = functions.init_browser(webdriver.Firefox(), 'rsilva', 'Savorthemom3nts')
    driver = LoginPage(page.driver).login(cpf, senha)
    
    if sapiens:
        
        sapiens.reset_driver(driver)
        
    else:
        
        sapiens = Page_sapiens(driver, {})
        
    return sapiens

### Cria/Carrega o Data Frame para exportar os dados

In [2]:
#database = pd.read_csv("registros_cadin.csv")

In [3]:
#database = database.dropna(subset=["CPF"])

In [5]:
#registros = {row[1]["CPF"]:{k:v for k,v in row[1].items()} for row in database.iterrows()}

In [5]:
sapiens = start_sapiens(CPF, PASS_2, None)

### Autenticar e Abrir planilhas Google

In [5]:
#gc = functions.authenticate_google("files/anatel.json")
#sht = gc.open(title='RA')

In [ ]:
#registros = sht.worksheet('Registros')
#registros = sht.get_worksheet(0)
#df_registros = gs_df.get_as_dataframe(registros, dtype=str)
#registros_rf = sht.worksheet('Registros_RF')
#df_registro_rf = gs_df.get_as_dataframe(registros_rf, dtype=str)
df = pd.DataFrame(columns=KEYS)

In [6]:
#processo = sht.worksheet("53504.005667/2018-21")
#df_processo = gs_df.get_as_dataframe(processo, dtype=str)
path = "files/"
expedidos = pd.read_csv(path+"Expedidos.csv", dtype=str)

In [7]:
expedidos.head()

,CPF,Nome,Fistel,Validade,End_Sede,End_Correspond.,Data_de_Nascimento,Logradouro,Número,Complemento,Bairro,Cidade,UF,Cep,Endereço_Completo,Processo,AR,Estado
0,28086615847,ALEXANDRE RIBEIRO COUTINHO,80106084674,17/05/2017,"RUA PROGRESSO, 21, casa 2\n- JARDIM SAO PAULO\...",RUA PITANGUEIRA 157 JD BARONESA \n CEP: 1310...,NaN,RUA FRANCISCO ANTONIO MIRANDA,88,NaN,EMERLINO MATARAZZO,SAO PAULO/SP,NaN,03809-130,RUA FRANCISCO ANTONIO MIRANDA 88 EMERLINO MAT...,53504.005890/2018-79,NaN,Excluído
1,27654632800,ANTONIO MOTA DE SANTANA,80106059050,18/05/2017,"RUA ADELINO MONAR, 170\n- SANTO ANTONIO\nCEP: ...",RUA ALFREDO FAVERO 262 JD PEDRO OMETTO \n CEP...,NaN,RUA ALBANO LAZIALE,33,NaN,JD. ARISE,SAO PAULO/SP,NaN,03574-040,RUA ALBANO LAZIALE 33 JD. ARISE \n CEP: 03574...,53504.005890/2018-79,NaN,Excluído
2,97628239820,CARLOS BARBOSA DOS SANTOS,80105984450,17/05/2017,"RUA JOSE NORBERTO PINTO, 317, casa\n- VILA ANA...",RESIDENCIAL RUA MAJOR JOSE INACIO 2553 CENTRO...,NaN,RUA MANOEL DOS SANTOS,321,NaN,PONTILHAO,CRUZEIRO/SP,NaN,12704-200,RUA MANOEL DOS SANTOS 321 PONTILHAO \n CEP: 1...,53504.005890/2018-79,NaN,Excluído
3,09617513803,CELSO PEREIRA,80106050770,17/05/2017,"RUA LUIZ MELGES, 25\n- CENTRO\nCEP: 17580000\n...",RUA SAO FRANCISCO 207 FDOS CENTRO \n CEP: 110...,NaN,RUA LUIZ MELGES,25,NaN,CENTRO,POMPEIA/SP,NaN,17580-000,RUA LUIZ MELGES 25 CENTRO \n CEP: 17580-000 P...,53504.005890/2018-79,NaN,Excluído
4,27003217821,FABIO RIBEIRO VAZ LOBO FILHO,80106068121,17/05/2017,"Rua Natal, 167, perto celsinho auto peças\n- V...",RUA MADALENA 83 CASA JD MARGARIDA \n CEP: 087...,NaN,RUA COSTA RICA,58,APARTAMENTO 2,JARDIM LAVINIA,MOCOCA/SP,NaN,13736-350,RUA COSTA RICA 58 APARTAMENTO 2 JARDIM LAVINIA...,53504.005890/2018-79,NaN,Excluído


In [10]:
processo = "53504.008813/2018-71"

slice_ = expedidos.Processo == processo

In [11]:
counter = 0

for i, row in expedidos[slice_].iterrows():
    
    cpf = row["CPF"].replace(".", "").replace('-', "")    
        
    if len(cpf) in (12, 13, 14):
        
        next
        
        while len(cpf) < 14:
            
            cpf = '0' + str(cpf)
            
    elif len(cpf) <= 11:
    
        while len(cpf) < 11:

            cpf = '0' + str(cpf)
            
    else:
        
        raise ValueError("CPF/CNPJ inválido {}".format(cpf))

    registro = sapiens.get_registro(cpf)    
   
    if not registro:
    
        sapiens.pesquisa_dados(cpf)
            
        registro = sapiens.get_registro(cpf)
        
        counter += 1
        
   
    if not (df["CPF"] == registro.get(cpf)).any():
        
        df = df.append(pd.Series(registro), ignore_index=True)
        
    if counter > 100:
        
        df.to_csv("registro_sapiens_devolvidos.csv", index=False)
        
        page = functions.init_browser(webdriver.Firefox(), 'rsilva', 'Savorthemom3nts')
        
        driver = LoginPage(page.driver).login(CPF, PASS_2)        
               
        sapiens.reset_driver(driver)
        
        sleep(10)        
                
        counter = 0          

KeyboardInterrupt: 

In [45]:
df = df.drop_duplicates()
df = df.fillna("")

In [46]:
df["Endereço_Completo"] = df.apply(lambda row: " ".join([str(row["Logradouro"]), 
                                                         str(row["Número"]), 
                                                         str(row["Complemento"]),
                                                         str(row["Bairro"]), "\n", "CEP:", 
                                                         str(row["Cep"]), 
                                                         str(row["Cidade"])]
                                                       ), axis=1
                                  )

In [65]:
#writer = pd.ExcelWriter("files/planilha_obito.xlsx")
#df.to_excel(writer, "consulta_RF", index=False, encoding='utf-8')
#writer.save()

In [129]:
#gs_df.set_with_dataframe(registros_rf, df, row=1, include_column_header=True)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Nome"]), row=1, col=9, include_column_header=False)
#gs_df.set_with_dataframe(registros, pd.DataFrame(df["Endereço_Completo"]), row=1, col=12, include_column_header=False)

In [48]:
gs_df.set_with_dataframe(registros, df, row=743, col=14, include_column_header=False)